In [2]:
from datasets import load_dataset
import tqdm
import pandas as pd
from transformers import pipeline
import random as rand
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from collections import OrderedDict
from transformers import logging

logging.set_verbosity_error()


In [3]:
USER_TAG = '<|Użytkownik|>: '
ASSISTANT_TAG = '<|Asystent|>: '
CUT_OFF_THRESHOLD = 0.85

### QA - Process Klej_dyk

In [4]:
klej_dyk = load_dataset("allegro/klej-dyk")
klej_dyk_df_train = klej_dyk['train'].to_pandas()
klej_dyk_df_test = klej_dyk['test'].to_pandas()
# combine train and test
klej_dyk_df = pd.concat([klej_dyk_df_train, klej_dyk_df_test], ignore_index=True)
klej_dyk_df = klej_dyk_df[klej_dyk_df["target"]==1][["question","answer"]]
klej_dyk_df["question"] = klej_dyk_df["question"].str.capitalize()
#### CREATE PROMPTS
klej_dyk_df["question"] = USER_TAG + klej_dyk_df["question"]
klej_dyk_df["answer"] =  ASSISTANT_TAG + klej_dyk_df["answer"]
#### PROMPTS
prompts_klej_dyk = klej_dyk_df["question"] + klej_dyk_df["answer"]
prompts_klej_dyk = pd.DataFrame(prompts_klej_dyk, columns=["text"]).reset_index(drop=True)
prompts_klej_dyk["trans_score"] = 1.0

### Process GPT-EXAM

In [5]:
gpt_exam = load_dataset("sdadas/gpt-exams")
gpt_exam_df = gpt_exam['train'].to_pandas()
#### CREATE PROMPTS
gpt_exam_df["question"] = USER_TAG + gpt_exam_df["question"]
gpt_exam_df["answer"] = ASSISTANT_TAG + gpt_exam_df["answer"]
#### PROMPTS
prompts_gpt_exam = gpt_exam_df["question"] + gpt_exam_df["answer"]
prompts_gpt_exam = pd.DataFrame(prompts_gpt_exam, columns=["text"]).reset_index(drop=True)
prompts_gpt_exam["trans_score"] = 1.0
prompts_gpt_exam[prompts_gpt_exam["text"].str.contains("Zagrajmy w grę fizz-buzz.")]


,text,trans_score


### Guanco

In [6]:
# oa_guanco = load_dataset("timdettmers/openassistant-guanaco")
# oa_guanco_df = oa_guanco['train'].to_pandas()

# pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", truncation=True, max_length=128)
# oa_guanco_en = []
# for i,text in enumerate(tqdm.tqdm(oa_guanco_df["text"])):
#     if pipe(text)[0]["label"] == "en":
#         oa_guanco_en.append(text)

# ### temp df
# df = pd.DataFrame(oa_guanco_en, columns=["text"])
# df = df[~df["text"].str.contains("Open Assistant")]
# df = df[~df["text"].str.contains("grammar")]
# df = df[~df["text"].str.contains("Re-write this sentence")]
# df = df[~df["text"].str.contains("Rewrite")]

# ## replace ### Asystent with <ASYSTENT_TAG>
# df["text"] = df["text"].str.replace("### Assistant", "<ASYSTENT_TAG>")
# ## replace ### Użytkownik with <USER_TAG>
# df["text"] = df["text"].str.replace("### Human", "<USER_TAG>")

# df.to_csv("../data/en/oa_guanco_en.csv", index=False)

## replace ### Asystent with <ASYSTENT_TAG>
# df["text"] = df["text"].str.replace("### Assistant", "<ASYSTENT_TAG>")
# ## replace ### Użytkownik with <USER_TAG>
# df["text"] = df["text"].str.replace("### Human", "<USER_TAG>")
# df.to_csv("../data/en/oa_guanco_en.csv", index=False)

# oa_translated_df = pd.read_csv("oa_dataset_translated.csv")
# oa_translated_df = oa_translated_df["translation"]
# ## create new column in new df named oa_prompts with translation
# oa_translated_df = oa_translated_df.str.replace("<ASYSTENT_TAG>", "### Asystent")
# prompts_oa = oa_translated_df.str.replace("<USER_TAG>", "### Użytkownik")

#### Dolly 15K

In [7]:
# dolly = load_dataset("databricks/databricks-dolly-15k")
# dolly_train_df = dolly['train'].to_pandas()
# # to csv
# dolly_train_df.to_csv("../data/en/dolly_train.csv", index=False)

### QA - with context - Process clarin-pl/poquad

In [8]:
poquad = load_dataset("clarin-pl/poquad")
poquad_df_train = poquad['train'].to_pandas()
poquad_df_test = poquad['validation'].to_pandas()
poquad_df = pd.concat([poquad_df_train, poquad_df_test], ignore_index=True)
poquad_df = poquad_df.drop_duplicates(subset=["title"]).reset_index(drop=True)
poquad_df["answers"] = poquad_df["answers"].apply(lambda x: x["text"][0])
poquad_df["answers"] = poquad_df["answers"].str.capitalize()
poquad_df = poquad_df[["context","question","answers"]]
poquad_df
#### CREATE PROMPTS
poquad_df["question_uzytownik"] = USER_TAG + poquad_df["question"]
poquad_df["answer_asystent"] = ASSISTANT_TAG + poquad_df["answers"]


### Process poleval_2019_mt

In [9]:
opus100 = load_dataset("opus100", "en-pl")
opus100_df = opus100['validation'].to_pandas()
#get data from dictonary inside column translation
opus100_df["en"] = opus100_df["translation"].apply(lambda x: x["en"])
opus100_df["pl"] = opus100_df["translation"].apply(lambda x: x["pl"])
## przetłumacz teksty z angielskiego na polski z pierwszego 1000 wierszy
# Take first 1000 rows
opus100_df_en_pl = opus100_df[:1000]
opus100_df_en_pl["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Angielskiego na Polski. " + opus100_df_en_pl["en"] + "### Asystent: " + opus100_df_en_pl["pl"]

opus100_df_pl_en = opus100_df[1000:2000]
opus100_df_pl_en["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Polskiego na Angielski. " + opus100_df["pl"] + "### Asystent: " + opus100_df["en"]

#### concat 
prompts_opus100 = pd.concat([opus100_df_en_pl["prompt"], opus100_df_pl_en["prompt"]], ignore_index=True)
# reset index
prompts_opus100 = prompts_opus100.reset_index(drop=True)


/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_19529/2628335791.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opus100_df_en_pl["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Angielskiego na Polski. " + opus100_df_en_pl["en"] + "### Asystent: " + opus100_df_en_pl["pl"]
/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_19529/2628335791.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opus100_df_pl_en["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Pols

# JOIN DATA

### JOIN dolly prompts

In [10]:
### Load translated dataset
dolly_translated_df = pd.read_csv("../../data/pl_eval/dolly_pl.csv")
# dolly_translated_df = dolly_translated_df[dolly_translated_df["trans_score_pl"] >= CUT_OFF_THRESHOLD]
dolly_translated_df["text"] = ''
# join prompts
for i, row in dolly_translated_df.iterrows():
    dolly_translated_df.at[i,"text"] = USER_TAG + row["instruction_pl"] + ' '+row["context_pl"] + ASSISTANT_TAG + row["response_pl"]
    # dolly_translated_df.at[i,"text"] = USER_TAG + row["instruction_pl"] + f"\n{synonimy_do_tekst_źródłowy[i%len(synonimy_do_tekst_źródłowy)]}\n" + row["context_pl"] + ASSISTANT_TAG + row["response_pl"]
# rename column to text
dolly_translated_df.reset_index(drop=True, inplace=True)
dolly_translated_df = dolly_translated_df[["text","trans_score_pl"]]
dolly_translated_df.columns = ["text","trans_score"]
dolly_translated_df
## find rows that have string <ASYSTENT_TAG> in them
dolly_translated_df[dolly_translated_df['text'].str.contains("<ASYSTENT_TAG>")]
dolly_translated_df[dolly_translated_df["text"].str.contains("Zagrajmy w grę fizz-buzz.")]



,text,trans_score


#### JOIN GUANCO PROMPTS

In [11]:
guanco_translated_df = pd.read_csv("../../data/pl_eval/guanco_pl.csv")

guanco_translated_df["text_pl"] = guanco_translated_df["text_pl"].str.replace("<ASYSTENT_TAG>: ", ASSISTANT_TAG)
guanco_translated_df["text_pl"] = guanco_translated_df["text_pl"].str.replace("<ASYSTENT_TAG>:", ASSISTANT_TAG)
guanco_translated_df["text_pl"] = guanco_translated_df["text_pl"].str.replace("<ASYSTENT_TAG>", ASSISTANT_TAG)
guanco_translated_df["text_pl"] = guanco_translated_df["text_pl"].str.replace("<USER_TAG>:", USER_TAG)
guanco_translated_df["text"] = guanco_translated_df["text_pl"].str.replace("<USER_TAG>: ", USER_TAG)
guanco_translated_df["text"] = guanco_translated_df["text"].str.replace("Open Assistant", "Krakowiak")
# guanco_translated_df = guanco_translated_df[guanco_translated_df["trans_score_pl"] >= CUT_OFF_THRESHOLD]["text"]
guanco_translated_df.reset_index(drop=True, inplace=True)
guanco_translated_df
guanco_translated_df = pd.DataFrame(guanco_translated_df, columns=["text","trans_score_pl"])
guanco_translated_df.columns = ["text","trans_score"]
guanco_translated_df



,text,trans_score
0,<|Użytkownik|>: Czy możesz napisać krótkie wp...,0.5953
1,<|Użytkownik|>: Jak mogę nauczyć się optymali...,0.7000
2,<|Użytkownik|>: Słuchałem symfonii “Nowy Świa...,0.7543
3,<|Użytkownik|>: Używam docker compose i muszę...,0.7683
4,<|Użytkownik|>: Tworzę backend NodeJS przy uż...,0.6548
...,...,...
2241,"<|Użytkownik|>: Napisz mi skrypt w pythonie, ...",0.7959
2242,<|Użytkownik|>: Jak używać modułów w pythonie...,0.5153
2243,"<|Użytkownik|>: Czy znasz Game Creator 2, atu...",0.8085
2244,<|Użytkownik|>: Jakie są główne czynniki wpły...,0.6401


In [ ]:
guanco_translated_df.iloc[106]["text"]

In [ ]:
# def remove_after_second_occurrence_from_right(input_string, substring):
#     parts = input_string.rsplit(substring, 2)
#     if len(parts) == 3:
#         result_string = parts[0] + substring + parts[1]
#         return result_string
#     else:
#         return input_string

# input_string = guanco_translated_df.iloc[5]
# substring = USER_TAG

# result = remove_after_second_occurrence_from_right(input_string, substring)
# print(result)


### JOIN OS WIZARD PROMPTS

In [13]:
os_wizard = pd.read_csv("../../data/pl_eval/oa_wizard_pl.csv")
os_wizard["text_pl"] = os_wizard["text_pl"].str.replace("<ASYSTENT_TAG>:", ASSISTANT_TAG)
os_wizard["text"] = os_wizard["text_pl"].str.replace("<USER_TAG>:", USER_TAG)
# trans_score_pl >= 0.8 get those rows
# os_wizard_translated_df = os_wizard[os_wizard["trans_score_pl"] >= CUT_OFF_THRESHOLD]["text"]
# os_wizard.reset_index(drop=True, inplace=True)
os_wizard_translated_df = os_wizard[["text","trans_score_pl"]]
os_wizard_translated_df.columns = ["text","trans_score"]


### Ultrachat

In [14]:
ultrachat_df = pd.read_csv("../../data/pl_eval/ultrachat_pl.csv")
## select rows wherte trans_score_pl >= 0.8
# ultrachat_translated_df = ultrachat_df[ultrachat_df["trans_score_pl"] >= CUT_OFF_THRESHOLD]["text_pl"].reset_index(drop=True)
# replace tag
ultrachat_df["text"] = ultrachat_df["text_pl"].str.replace("<ASSISTANT_TAG>: ", ASSISTANT_TAG)
ultrachat_df["text"] = ultrachat_df["text"].str.replace("<USER_TAG>: ", USER_TAG)
ultrachat_translated_df = ultrachat_df[["text","trans_score_pl"]]
ultrachat_translated_df.columns = ["text","trans_score"]
ultrachat_translated_df

,text,trans_score
0,<|Użytkownik|>: Czy istnieją szczególne korzyś...,0.8210
1,<|Użytkownik|>: Jaki procent powierzchni Ziemi...,0.8788
2,<|Użytkownik|>: Jaka jest obecnie najpopularni...,0.8619
3,<|Użytkownik|>: Czy mindful walking może być u...,0.8212
4,<|Użytkownik|>: Jak ten podręcznik wypada w po...,0.8555
...,...,...
135160,<|Użytkownik|>: Biorąc pod uwagę tekst: Piękna...,0.8160
135161,<|Użytkownik|>: Żadna z tych postaci nie należ...,0.8222
135162,<|Użytkownik|>: Proszę sparafrazować linijkę “...,0.7902
135163,<|Użytkownik|>: Edukacyjne i interaktywne kosz...,0.8551


In [15]:
ultrachat_translated_df.iloc[rand.randint(0, len(ultrachat_translated_df))]

text           <|Użytkownik|>: Które księgi Biblii są uważane...
trans_score                                               0.8508
Name: 57776, dtype: object

In [16]:
prompts_klej_dyk.iloc[0]

text           <|Użytkownik|>: Jakie choroby genetyczne dzied...
trans_score                                                  1.0
Name: 0, dtype: object

In [17]:
prompts_gpt_exam.iloc[0]

text           <|Użytkownik|>: Kim był Donatello i jakie są j...
trans_score                                                  1.0
Name: 0, dtype: object

In [ ]:
prompts_opus100.iloc[0]

In [ ]:
os_wizard_translated_df.iloc[0]

In [ ]:
dolly_translated_df.iloc[0]

In [ ]:
ultrachat_translated_df.iloc[0]

In [18]:
## concat all prompts
prompts = pd.concat([prompts_klej_dyk, prompts_gpt_exam, os_wizard_translated_df, guanco_translated_df, dolly_translated_df, ultrachat_translated_df], ignore_index=True)
## shuffle rows in prompts 
prompts = prompts.sample(frac=1, random_state=1).reset_index(drop=True)
## add column NAMED text
prompts["text"] = prompts["text"].str.replace("<USER_TAG>:", USER_TAG)
prompts["text"] = prompts["text"].str.replace("<ASSISTANT_TAG>", ASSISTANT_TAG)

# prompts = pd.DataFrame(prompts, columns=["text","trans_score"])
prompts

,text,trans_score
0,<|Użytkownik|>: Które narodowe drużyny rugby o...,0.8786
1,<|Użytkownik|>: Ile jaj zazwyczaj składa klaun...,0.8697
2,<|Użytkownik|>: Czy możesz polecić rodzaj muzy...,0.8617
3,<|Użytkownik|>: Jakie są długoterminowe perspe...,0.8396
4,<|Użytkownik|>: Jakie są najpopularniejsze alg...,1.0000
...,...,...
159015,<|Użytkownik|>: Jak długo zazwyczaj trwa dołąc...,0.7560
159016,<|Użytkownik|>: Czy konieczne jest posiadanie ...,0.8245
159017,<|Użytkownik|>: W jaki sposób Sahitya Akademi ...,0.8395
159018,<|Użytkownik|>: Jakie są główne elementy rosyj...,1.0000


In [19]:
# prompts.iloc[44]['text'].split(ASSISTANT_TAG)
# prompts.iloc[44]['text'].split(USER_TAG)
for i, row in prompts.iterrows():
    try:
        prompts.iloc[i]['text'].split(ASSISTANT_TAG)[0] + ASSISTANT_TAG + prompts.iloc[i]['text'].split(USER_TAG)[1]
    except:
        print(i)

In [ ]:
# prompts.iloc[45]['text'].split(ASSISTANT_TAG)[1]
prompts.iloc[45]['text'].split(ASSISTANT_TAG)[0].replace(USER_TAG, "")

In [ ]:
prompts.iloc[45]

In [ ]:
prompts.iloc[103101]["text"]

In [ ]:
init_prompts

In [20]:
init_prompts = prompts
bad_examples = []
arr = []
scores = []
for i in tqdm.tqdm(range(0, len(init_prompts)), total=len(init_prompts)):
    try:
        user_quotes = OrderedDict()
        user_quotes["role"] = "user"
        user_quotes["content"] = prompts.iloc[i]['text'].split(ASSISTANT_TAG)[0].replace(USER_TAG, "")
        ###
        assistant_quotes =  OrderedDict()
        assistant_quotes["role"] = "assistant"
        assistant_quotes["content"] = prompts.iloc[i]['text'].split(ASSISTANT_TAG)[1]

        prompt = [user_quotes, assistant_quotes]
        arr.append(prompt)
        scores.append(prompts.iloc[i]['trans_score'])
    except:
        bad_examples.append(i)

full_prompts = pd.DataFrame([i]*len(arr), columns=["text"])
full_prompts["text"] = arr
print(len(full_prompts), len(bad_examples))

100%|██████████| 159020/159020 [00:08<00:00, 19484.02it/s]

158911 109


### Post to huggingface dataset

In [21]:
full_prompts = pd.DataFrame.from_dict({"text":arr,"trans_score":scores})

In [ ]:
full_prompts

In [22]:
## select text where trans_score >= 0.8
prompts_filtered = full_prompts[full_prompts["trans_score"] >= 0.85].reset_index(drop=True)

In [25]:
prompts_filtered.iloc[rand.randint(0, len(prompts_filtered))]["text"]

[OrderedDict([('role', 'user'),
              ('content',
               ' Wygeneruj listę sześciu rzeczy, które przynoszą Ci radość.\r\n')]),
 OrderedDict([('role', 'assistant'),
              ('content',
               ' 1. Spędzanie czasu z rodziną i przyjaciółmi\r\n2. Słuchanie muzyki i taniec\r\n3. Czytanie dobrej książki\r\n4. Będąc w przyrodzie, czy to wędrówki, kemping, czy po prostu spacer\r\n5. Gotowanie i wypróbowywanie nowych przepisów\r\n6. Podróżuj i poznawaj nowe kultury i potrawy')])]

In [ ]:
# ### get random 1000 rows from prompts with seed and put into validation df
# prompts_sample = full_prompts.sample(n=1000, random_state=1)
# prompts_sample = full_prompts.reset_index(drop=True)

In [26]:
train = prompts_filtered[:len(prompts_filtered)-500].reset_index(drop=True)
valid = prompts_filtered[len(prompts_filtered)-500:].reset_index(drop=True)

In [27]:
train

,text,trans_score
0,"[{'role': 'user', 'content': 'Które narodowe d...",0.8786
1,"[{'role': 'user', 'content': 'Ile jaj zazwycza...",0.8697
2,"[{'role': 'user', 'content': 'Czy możesz polec...",0.8617
3,"[{'role': 'user', 'content': 'Jakie są najpopu...",1.0000
4,"[{'role': 'user', 'content': 'Napisz osobisty ...",0.8737
...,...,...
50329,"[{'role': 'user', 'content': 'Użyj co najmniej...",0.8548
50330,"[{'role': 'user', 'content': 'Czy etyczne jest...",0.8570
50331,"[{'role': 'user', 'content': 'Jak oceniasz, kt...",0.8554
50332,"[{'role': 'user', 'content': 'Czy różne uczeln...",0.8542


In [28]:
valid.iloc[rand.randint(0, len(valid))]["text"]

[OrderedDict([('role', 'user'),
              ('content', '6. Śledzenie zamówień i historia')]),
 OrderedDict([('role', 'assistant'),
              ('content',
               'Nie mogę wykonać tej funkcji, ponieważ nie mam możliwości śledzenia zamówień ani przechowywania danych historycznych.')])]

In [ ]:
import pyarrow as pa

In [ ]:
table = pa.Table.from_pandas(train,)

In [29]:
ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train)
ds['validation'] = Dataset.from_pandas(valid)
ds['everything'] = Dataset.from_pandas(full_prompts)

In [31]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'trans_score'],
        num_rows: 50334
    })
    validation: Dataset({
        features: ['text', 'trans_score'],
        num_rows: 500
    })
    everything: Dataset({
        features: ['text', 'trans_score'],
        num_rows: 158911
    })
})

In [32]:
ds.push_to_hub("szymonrucinski/krakowiak-pl")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  4.35it/s]


In [ ]:
from datasets import load_dataset

In [ ]:
x = load_dataset("szymonrucinski/krakowiak-pl")

In [41]:
from transformers import AutoTokenizer
base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=2048)


In [42]:
["train"]["text"][2]

<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_19529/210924293.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ["train"]["text"][2]
/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_19529/210924293.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ["train"]["text"][2]
/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_19529/210924293.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ["train"]["text"][2]
/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_19529/210924293.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ["train"]["text"][2]


TypeError: list indices must be integers or slices, not str

In [47]:
tokenizer.apply_chat_template(ds["train"]["text"][2], tokenize=False)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


'<s>[INST] Czy możesz polecić rodzaj muzyki, który idealnie nadaje się do nauki lub pracy? [/INST]Jako model języka sztucznej inteligencji nie mam osobistych preferencji, ale mogę zasugerować muzykę instrumentalną i klasyczną, która jest uważana za idealną do nauki i pracy. Niektóre popularne opcje obejmują muzykę ambientową, jazz instrumentalny, muzykę klasyczną i dźwięki natury. Te rodzaje muzyki są kojące i pomagają skupić się, skoncentrować i zmniejszyć stres podczas pracy lub nauki.</s> '

In [ ]:
def process_for

In [ ]:
x["train"]["text"]

In [46]:
tokenizer.use_default_system_prompt = False
d_dict = {"train":[], 'valid':[]}
tokenizer.pad_token = tokenizer.eos_token

for stage in d_dict.keys():
    for instruction in ds[stage]["text"]:
        try:
            d_dict["train"].append(tokenizer.apply_chat_template(instruction, tokenize=False))
        except:
            pass
            


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


KeyError: 'valid'

In [ ]:
chat["train"]["text"]